In [1]:
import pandas as pd
import joblib
import numpy as np

import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
# 카테고리형 변수들을 수치형으로 변경하는 함수 생성
from sklearn.preprocessing import LabelEncoder


def cat_encoding(df):
    le = LabelEncoder()
    original_columns = list(df.columns)
    
    for col in df:
        # df의 컬럼의 유형이 object인 것들만
        if df[col].dtype == 'object':
            
            # df의 컬럼별로 포함된 개체 수가 2개 이하이면 LabelEncoding
            if len(list(df[col].unique())) <= 2:
                le.fit(df[col])
                df[col] = le.transform(df[col])

            # df의 컬럼별로 포함된 개체 수가 3개 이상이면 ont-hot-encoding
            elif len(list(df[col].unique())) > 2:            
                df = pd.get_dummies(df, columns = [col], dummy_na=False)
    # 새롭게 만들어진 컬럼들의 이름을 리스트로 저장
    new_columns = [c for c in df.columns if c not in original_columns]
    
    # 수치형으로 변경된 df와 새롭게 만들어진 컬럼 이름 리스트를 반환
    return df, new_columns

In [3]:
# 파일 로드
bur_df = pd.read_csv('../../data/bureau.csv')
bb_df = pd.read_csv('../../data/bureau_balance.csv')

In [4]:
# bb_df 에 대한 결측값 확인
mis_val = bb_df.isnull().sum()
mis_val

SK_ID_BUREAU      0
MONTHS_BALANCE    0
STATUS            0
dtype: int64

In [5]:
# bureau_balance 파일에서 가져온 df의 category변수들을 수치형으로 변경
bb_df, bb_new_columns = cat_encoding(bb_df)

In [6]:
# months_balance(수치형) 컬럼에 대해 agg 기준 설정
bb_aggregations = {'MONTHS_BALANCE': ['min', 'max', 'mean', 'median', 'sum', 'size']}

In [7]:
# 새롭게 만들어진 컬럼들을 평균과 합계(status_0의 개수)로 agg
for col in bb_new_columns:
    bb_aggregations[col] = ['mean', 'sum']

In [8]:
# SK_ID_BUREAU별로 groupby
bb_agg = bb_df.groupby('SK_ID_BUREAU').agg(bb_aggregations)

In [9]:
# 컬럼구분 및 이름 변경
bb_agg.columns = pd.Index([e[0] + "_" + e[1].upper() for e in bb_agg.columns.tolist()])

In [10]:
bb_agg

,MONTHS_BALANCE_MIN,MONTHS_BALANCE_MAX,MONTHS_BALANCE_MEAN,MONTHS_BALANCE_MEDIAN,MONTHS_BALANCE_SUM,MONTHS_BALANCE_SIZE,STATUS_0_MEAN,STATUS_0_SUM,STATUS_1_MEAN,STATUS_1_SUM,...,STATUS_3_MEAN,STATUS_3_SUM,STATUS_4_MEAN,STATUS_4_SUM,STATUS_5_MEAN,STATUS_5_SUM,STATUS_C_MEAN,STATUS_C_SUM,STATUS_X_MEAN,STATUS_X_SUM
SK_ID_BUREAU,,,,,,,,,,,,,,,,,,,,,
5001709,-96,0,-48.0,-48.0,-4656,97,0.000000,0,0.000000,0,...,0.0,0,0.0,0,0.0,0,0.886598,86,0.113402,11
5001710,-82,0,-41.0,-41.0,-3403,83,0.060241,5,0.000000,0,...,0.0,0,0.0,0,0.0,0,0.578313,48,0.361446,30
5001711,-3,0,-1.5,-1.5,-6,4,0.750000,3,0.000000,0,...,0.0,0,0.0,0,0.0,0,0.000000,0,0.250000,1
5001712,-18,0,-9.0,-9.0,-171,19,0.526316,10,0.000000,0,...,0.0,0,0.0,0,0.0,0,0.473684,9,0.000000,0
5001713,-21,0,-10.5,-10.5,-231,22,0.000000,0,0.000000,0,...,0.0,0,0.0,0,0.0,0,0.000000,0,1.000000,22
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6842884,-47,0,-23.5,-23.5,-1128,48,0.187500,9,0.000000,0,...,0.0,0,0.0,0,0.0,0,0.416667,20,0.395833,19
6842885,-23,0,-11.5,-11.5,-276,24,0.500000,12,0.000000,0,...,0.0,0,0.0,0,0.5,12,0.000000,0,0.000000,0
6842886,-32,0,-16.0,-16.0,-528,33,0.242424,8,0.000000,0,...,0.0,0,0.0,0,0.0,0,0.757576,25,0.000000,0


In [11]:
# bureau.csv 파일에서 가져온 df에 SK_ID_BUREAU기준으로 join
bur_df = bur_df.join(bb_agg, how='left', on='SK_ID_BUREAU')

In [12]:
# 더이상 필요하지 않은 SK_ID_BUREAU 컬럼 drop
bur_df.drop(['SK_ID_BUREAU'], axis=1, inplace= True)

In [13]:
bur_df

,SK_ID_CURR,CREDIT_ACTIVE,CREDIT_CURRENCY,DAYS_CREDIT,CREDIT_DAY_OVERDUE,DAYS_CREDIT_ENDDATE,DAYS_ENDDATE_FACT,AMT_CREDIT_MAX_OVERDUE,CNT_CREDIT_PROLONG,AMT_CREDIT_SUM,...,STATUS_3_MEAN,STATUS_3_SUM,STATUS_4_MEAN,STATUS_4_SUM,STATUS_5_MEAN,STATUS_5_SUM,STATUS_C_MEAN,STATUS_C_SUM,STATUS_X_MEAN,STATUS_X_SUM
0,215354,Closed,currency 1,-497,0,-153.0,-153.0,NaN,0,91323.00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,215354,Active,currency 1,-208,0,1075.0,NaN,NaN,0,225000.00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,215354,Active,currency 1,-203,0,528.0,NaN,NaN,0,464323.50,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,215354,Active,currency 1,-203,0,NaN,NaN,NaN,0,90000.00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,215354,Active,currency 1,-629,0,1197.0,NaN,77674.5,0,2700000.00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1716423,259355,Active,currency 1,-44,0,-30.0,NaN,0.0,0,11250.00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1716424,100044,Closed,currency 1,-2648,0,-2433.0,-2493.0,5476.5,0,38130.84,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1716425,100044,Closed,currency 1,-1809,0,-1628.0,-970.0,NaN,0,15570.00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1716426,246829,Closed,currency 1,-1878,0,-1513.0,-1513.0,NaN,0,36000.00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [14]:
# 결측치 확인
bur_df.isnull().sum().sort_values(ascending=False)

AMT_ANNUITY               1226791
AMT_CREDIT_MAX_OVERDUE    1124488
STATUS_X_SUM               942074
STATUS_1_MEAN              942074
MONTHS_BALANCE_MIN         942074
MONTHS_BALANCE_MAX         942074
STATUS_X_MEAN              942074
MONTHS_BALANCE_MEDIAN      942074
MONTHS_BALANCE_SUM         942074
MONTHS_BALANCE_SIZE        942074
STATUS_0_MEAN              942074
STATUS_0_SUM               942074
MONTHS_BALANCE_MEAN        942074
STATUS_1_SUM               942074
STATUS_2_SUM               942074
STATUS_3_MEAN              942074
STATUS_3_SUM               942074
STATUS_4_MEAN              942074
STATUS_4_SUM               942074
STATUS_5_MEAN              942074
STATUS_5_SUM               942074
STATUS_C_MEAN              942074
STATUS_C_SUM               942074
STATUS_2_MEAN              942074
DAYS_ENDDATE_FACT          633653
AMT_CREDIT_SUM_LIMIT       591780
AMT_CREDIT_SUM_DEBT        257669
DAYS_CREDIT_ENDDATE        105553
AMT_CREDIT_SUM                 13
DAYS_CREDIT   

In [15]:
# bur_df 또한 category변수들을 수치형으로 변환
bur_df, bur_new_columns = cat_encoding(bur_df)

In [13]:
# 결측값이 50%가 넘어가는 변수들은 제외

# bur_df.drop(['AMT_CREDIT_MAX_OVERDUE'], axis=1, inplace= True)
# bur_df.drop(['AMT_ANNUITY'], axis=1, inplace= True)

In [16]:
# 실제 낸 날짜 - 예정 날짜
bur_df['DAYS_INADVANCE'] = bur_df['DAYS_ENDDATE_FACT'] - bur_df['DAYS_CREDIT_ENDDATE']

# 신용거래 기간(거래완료 예정날짜 - 신청날짜) 
bur_df['CREDIT_DURATION'] = bur_df['DAYS_CREDIT_ENDDATE'] - bur_df['DAYS_CREDIT']

# 부채 비율
bur_df['DEBT_PRECENTAGE'] = bur_df['AMT_CREDIT_SUM_DEBT'] / bur_df['AMT_CREDIT_SUM']

# 거래 총액 - 총 부채액(갚은 돈(?))
bur_df['AMT_CREDIT_DIFF'] = bur_df['AMT_CREDIT_SUM'] - bur_df['AMT_CREDIT_SUM_DEBT']

# 월별 갚는 비율
bur_df['MONTH_CREDIT_REPAY_RATIO'] =  bur_df['AMT_ANNUITY'] / bur_df['AMT_CREDIT_SUM'] 

# 현재 거래 진행 이후 업데이트 됐을 때까지의 날짜
bur_df['DAYS_CREDIT_NEW'] = bur_df['DAYS_CREDIT_UPDATE'] - bur_df['DAYS_CREDIT']

# 총 거래액에서 갚지못한 돈의 최대값 비율
bur_df['AMT_CREDIT_OVERDUE_RATIO'] = bur_df['AMT_CREDIT_MAX_OVERDUE']/ bur_df['AMT_CREDIT_SUM'] 

In [17]:
# 새로운 변수들 생성 과정에서 만들어진 무한대 값을 가진 행을 제외
bur_df = bur_df[bur_df['AMT_CREDIT_OVERDUE_RATIO'] != np.inf]
bur_df = bur_df[bur_df['DEBT_PRECENTAGE'] != np.inf]
bur_df = bur_df[bur_df['DEBT_PRECENTAGE'] != -np.inf]
bur_df = bur_df[bur_df['MONTH_CREDIT_REPAY_RATIO'] != np.inf]

In [18]:
bur_df[bur_df.isin([np.inf]).any(1)].count().sort_values(ascending=False)

AMT_CREDIT_OVERDUE_RATIO    0
STATUS_C_MEAN               0
STATUS_5_MEAN               0
STATUS_4_SUM                0
STATUS_4_MEAN               0
                           ..
CREDIT_ACTIVE_Closed        0
CREDIT_ACTIVE_Bad debt      0
CREDIT_ACTIVE_Active        0
STATUS_X_SUM                0
SK_ID_CURR                  0
Length: 65, dtype: int64

In [19]:
bur_df

,SK_ID_CURR,DAYS_CREDIT,CREDIT_DAY_OVERDUE,DAYS_CREDIT_ENDDATE,DAYS_ENDDATE_FACT,AMT_CREDIT_MAX_OVERDUE,CNT_CREDIT_PROLONG,AMT_CREDIT_SUM,AMT_CREDIT_SUM_DEBT,AMT_CREDIT_SUM_LIMIT,...,CREDIT_TYPE_Mortgage,CREDIT_TYPE_Real estate loan,CREDIT_TYPE_Unknown type of loan,DAYS_INADVANCE,CREDIT_DURATION,DEBT_PRECENTAGE,AMT_CREDIT_DIFF,MONTH_CREDIT_REPAY_RATIO,DAYS_CREDIT_NEW,AMT_CREDIT_OVERDUE_RATIO
0,215354,-497,0,-153.0,-153.0,NaN,0,91323.00,0.0,NaN,...,0,0,0,0.0,344.0,0.00000,91323.00,NaN,366,NaN
1,215354,-208,0,1075.0,NaN,NaN,0,225000.00,171342.0,NaN,...,0,0,0,NaN,1283.0,0.76152,53658.00,NaN,188,NaN
2,215354,-203,0,528.0,NaN,NaN,0,464323.50,NaN,NaN,...,0,0,0,NaN,731.0,NaN,NaN,NaN,187,NaN
3,215354,-203,0,NaN,NaN,NaN,0,90000.00,NaN,NaN,...,0,0,0,NaN,NaN,NaN,NaN,NaN,187,NaN
4,215354,-629,0,1197.0,NaN,77674.5,0,2700000.00,NaN,NaN,...,0,0,0,NaN,1826.0,NaN,NaN,NaN,608,0.028768
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1716423,259355,-44,0,-30.0,NaN,0.0,0,11250.00,11250.0,0.0,...,0,0,0,NaN,14.0,1.00000,0.00,NaN,25,0.000000
1716424,100044,-2648,0,-2433.0,-2493.0,5476.5,0,38130.84,0.0,0.0,...,0,0,0,-60.0,215.0,0.00000,38130.84,NaN,155,0.143624
1716425,100044,-1809,0,-1628.0,-970.0,NaN,0,15570.00,NaN,NaN,...,0,0,0,658.0,181.0,NaN,NaN,NaN,842,NaN
1716426,246829,-1878,0,-1513.0,-1513.0,NaN,0,36000.00,0.0,0.0,...,0,0,0,0.0,365.0,0.00000,36000.00,NaN,370,NaN


In [ ]:
# category 변수들로 인해 새롭게 생성된 수치형 변수들에 대해 agg 기준 설정

cat_aggregations = {}
values = ['min', 'max', 'mean', 'median', 'sum', 'size']

# bb_df에서 새롭게 생성된 변수들
for cat in bb_new_columns: cat_aggregations[cat + "_MEAN"] = ['mean']
for cat in bb_new_columns: cat_aggregations[cat + "_SUM"] = ['mean', 'sum', 'max', 'min']

for value in values: 
    cat_aggregations["MONTHS_BALANCE_" + value.upper()] = ['mean'] 
    
# bur_df에서 새롭게 생성된 변수들
for cat in bur_new_columns: cat_aggregations[cat] = ['mean', 'sum']
    
    
cat_aggregations

In [23]:
# 수치형 변수들에 대한 agg 기준 설정

# category형 변수들의 컬럼이름을 리스트로 저장
categories = []
for key, value in cat_aggregations.items():
    categories.append(key)

num_aggregations = {}

# category형 변수들의 컬럼이름을 제외한 컬럼들에 대해 agg 기준 설정
num_columns = [ _ for _ in bur_df.columns if _ not in categories]
num_columns.remove('SK_ID_CURR')
for col in num_columns:
    num_aggregations[col] = ['min', 'max', 'mean', 'median', 'sum', 'size']
    
num_aggregations

{'DAYS_CREDIT': ['min', 'max', 'mean', 'median', 'sum', 'size'],
 'CREDIT_DAY_OVERDUE': ['min', 'max', 'mean', 'median', 'sum', 'size'],
 'DAYS_CREDIT_ENDDATE': ['min', 'max', 'mean', 'median', 'sum', 'size'],
 'DAYS_ENDDATE_FACT': ['min', 'max', 'mean', 'median', 'sum', 'size'],
 'AMT_CREDIT_MAX_OVERDUE': ['min', 'max', 'mean', 'median', 'sum', 'size'],
 'CNT_CREDIT_PROLONG': ['min', 'max', 'mean', 'median', 'sum', 'size'],
 'AMT_CREDIT_SUM': ['min', 'max', 'mean', 'median', 'sum', 'size'],
 'AMT_CREDIT_SUM_DEBT': ['min', 'max', 'mean', 'median', 'sum', 'size'],
 'AMT_CREDIT_SUM_LIMIT': ['min', 'max', 'mean', 'median', 'sum', 'size'],
 'AMT_CREDIT_SUM_OVERDUE': ['min', 'max', 'mean', 'median', 'sum', 'size'],
 'DAYS_CREDIT_UPDATE': ['min', 'max', 'mean', 'median', 'sum', 'size'],
 'AMT_ANNUITY': ['min', 'max', 'mean', 'median', 'sum', 'size'],
 'DAYS_INADVANCE': ['min', 'max', 'mean', 'median', 'sum', 'size'],
 'CREDIT_DURATION': ['min', 'max', 'mean', 'median', 'sum', 'size'],
 'DEBT

In [24]:
# 위에서 생성한 agg 조건에 대해 SK_ID_CURR을 기준으로 groupby 
bureau_agg = bur_df.groupby('SK_ID_CURR').agg({**num_aggregations, **cat_aggregations})

In [25]:
# 변수명 정리
bureau_agg.columns = pd.Index(['BUR_' + e[0] + "_" + e[1].upper() for e in bureau_agg.columns.tolist()])

In [26]:
# 확인
bureau_agg

,BUR_DAYS_CREDIT_MIN,BUR_DAYS_CREDIT_MAX,BUR_DAYS_CREDIT_MEAN,BUR_DAYS_CREDIT_MEDIAN,BUR_DAYS_CREDIT_SUM,BUR_DAYS_CREDIT_SIZE,BUR_CREDIT_DAY_OVERDUE_MIN,BUR_CREDIT_DAY_OVERDUE_MAX,BUR_CREDIT_DAY_OVERDUE_MEAN,BUR_CREDIT_DAY_OVERDUE_MEDIAN,...,BUR_CREDIT_TYPE_Microloan_MEAN,BUR_CREDIT_TYPE_Microloan_SUM,BUR_CREDIT_TYPE_Mobile operator loan_MEAN,BUR_CREDIT_TYPE_Mobile operator loan_SUM,BUR_CREDIT_TYPE_Mortgage_MEAN,BUR_CREDIT_TYPE_Mortgage_SUM,BUR_CREDIT_TYPE_Real estate loan_MEAN,BUR_CREDIT_TYPE_Real estate loan_SUM,BUR_CREDIT_TYPE_Unknown type of loan_MEAN,BUR_CREDIT_TYPE_Unknown type of loan_SUM
SK_ID_CURR,,,,,,,,,,,,,,,,,,,,,
100001,-1572,-49,-735.000000,-857.0,-5145,7,0,0,0.0,0.0,...,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0
100002,-1437,-103,-874.000000,-1042.5,-6992,8,0,0,0.0,0.0,...,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0
100003,-2586,-606,-1400.750000,-1205.5,-5603,4,0,0,0.0,0.0,...,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0
100004,-1326,-408,-867.000000,-867.0,-1734,2,0,0,0.0,0.0,...,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0
100005,-373,-62,-190.666667,-137.0,-572,3,0,0,0.0,0.0,...,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
456249,-2713,-483,-1667.076923,-1680.0,-21672,13,0,0,0.0,0.0,...,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0
456250,-1002,-760,-862.000000,-824.0,-2586,3,0,0,0.0,0.0,...,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0
456253,-919,-713,-867.500000,-919.0,-3470,4,0,0,0.0,0.0,...,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0


In [27]:
# 결측값 확인
bureau_agg.isnull().sum().sort_values(ascending=False).head(50)

BUR_MONTH_CREDIT_REPAY_RATIO_MAX       188148
BUR_MONTH_CREDIT_REPAY_RATIO_MIN       188148
BUR_MONTH_CREDIT_REPAY_RATIO_MEAN      188148
BUR_MONTH_CREDIT_REPAY_RATIO_MEDIAN    188148
BUR_AMT_ANNUITY_MEDIAN                 187750
BUR_AMT_ANNUITY_MEAN                   187750
BUR_AMT_ANNUITY_MAX                    187750
BUR_AMT_ANNUITY_MIN                    187750
BUR_STATUS_3_SUM_MAX                   171211
BUR_STATUS_5_SUM_MEAN                  171211
BUR_STATUS_4_SUM_MIN                   171211
BUR_STATUS_4_SUM_MAX                   171211
BUR_STATUS_3_SUM_MIN                   171211
BUR_STATUS_4_SUM_MEAN                  171211
BUR_STATUS_3_SUM_MEAN                  171211
BUR_STATUS_5_SUM_MIN                   171211
BUR_STATUS_2_SUM_MIN                   171211
BUR_STATUS_2_SUM_MAX                   171211
BUR_STATUS_2_SUM_MEAN                  171211
BUR_STATUS_5_SUM_MAX                   171211
BUR_MONTHS_BALANCE_MEAN_MEAN           171211
BUR_MONTHS_BALANCE_SIZE_MEAN      

In [28]:
# joblib파일로 저장
joblib.dump(filename= "bur_df_nodrop_2.joblib", value=bureau_agg)

['bur_agg_nodrop.joblib']